In [1]:
import gc
import json
import os
import math
import multiprocessing
import numpy as np
import pandas as pd
import torch
import importlib
import logging
from pathlib import Path
from sklearn.model_selection import GroupKFold, GroupShuffleSplit

# Pycox and PyTorch tuples for survival analysis
import torchtuples as tt
import pycox
from pycox.preprocessing.label_transforms import LabTransDiscreteTime
from pycox.models import CoxPH, DeepHit
from pycox.evaluation import EvalSurv

# Ray for hyperparameter tuning and distributed processing
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.search.bayesopt import BayesOptSearch
from ray.tune.search.optuna import OptunaSearch
from ray.tune.search import ConcurrencyLimiter
from ray.tune.schedulers import ASHAScheduler, PopulationBasedTraining
from ray.air import session
import ray.cloudpickle as pickle

# Custom modules for data handling, balancing, training, evaluation, and model architectures
import dataloader2
import databalancer2
import datatrainer2
import modeleval
import netweaver2

# Reload custom modules to ensure latest changes are available
importlib.reload(dataloader2)
importlib.reload(databalancer2)
importlib.reload(datatrainer2)
importlib.reload(modeleval)
importlib.reload(netweaver2)

# Import specific functions from custom modules to keep code clean and readable
from netweaver2 import (
    lstm_net_init, DHANNWrapper, LSTMWrapper, generalized_ann_net_init
)
from dataloader2 import (
    load_and_transform_data, preprocess_data #stack_sequences, dh_dataset_loader
)
from databalancer2 import (
    define_medoid_general, df_event_focus, rebalance_data, underbalance_data_general, medoid_cluster, 
    dh_rebalance_data
)
from datatrainer2 import (
    recursive_clustering, prepare_training_data, 
    prepare_validation_data, lstm_training
)
from modeleval import (
    dh_test_model, nam_dagostino_chi2, get_baseline_hazard_at_timepoints, combined_test_model
)

import psutil
torch.cuda.empty_cache()
gc.collect()

101

In [8]:
# Define Constants and Load Datasets
RANDOM_SEED = 12345
N_SPLIT = 2
FEATURE_COLS = ['gender', 'dm', 'ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3']
DURATION_COL = 'date_from_sub_60'
EVENT_COL = 'endpoint'
CLUSTER_COL = 'key'
TIME_GRID = np.array([i * 365 for i in range(6)])

# Define Feature Groups
CAT_FEATURES = ['gender', 'dm', 'ht', 'sprint']
LOG_FEATURES = ['a1c', 'po4', 'UACR_mg_g', 'Cr']
STANDARD_FEATURES = ['age', 'alb', 'ca', 'hb', 'hco3']
PASSTHROUGH_FEATURES = ['key', 'date_from_sub_60', 'endpoint']

# Load and Transform Data
BASE_FILENAME = '/mnt/d/pydatascience/g3_regress/data/X/X_20240628'
X_train_transformed, X_test_transformed = load_and_transform_data(
    BASE_FILENAME, CAT_FEATURES, LOG_FEATURES, STANDARD_FEATURES, PASSTHROUGH_FEATURES
)

2024-11-16 00:20:21,540 - INFO - Transforming training data...
2024-11-16 00:20:35,024 - INFO - Transforming test data...


In [24]:
from lifelines import CoxPHFitter

# Convert all non-target events to 0 (censored)
X_train_transformed["event1"] = X_train_transformed[EVENT_COL].apply(lambda x: 1 if x == 1 else 0)
X_train_transformed["event2"] = X_train_transformed[EVENT_COL].apply(lambda x: 1 if x == 2 else 0)

class_counts = X_train_transformed[EVENT_COL].value_counts()
X_train_transformed['weights'] = X_train_transformed[EVENT_COL].map(lambda e: 1 / class_counts[e]).values

# Fit a Cox model for each event type
cox_model_event_1 = CoxPHFitter()
cox_model_event_1.fit(X_train_transformed[FEATURE_COLS + [DURATION_COL, 'event1', CLUSTER_COL, 'weights']], duration_col=DURATION_COL, event_col="event1", cluster_col=CLUSTER_COL, weights_col="weights")

cox_model_event_2 = CoxPHFitter()
cox_model_event_2.fit(X_train_transformed[FEATURE_COLS + [DURATION_COL, 'event2', CLUSTER_COL, 'weights']], duration_col=DURATION_COL, event_col="event2", cluster_col=CLUSTER_COL, weights_col="weights")



/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/coxph_fitter.py:1356: StatisticalWarning: It appears your weights are not integers, possibly propensity or sampling scores then?
It's important to know that the naive variance estimates of the coefficients are biased. Instead a) set `robust=True` in the call to `fit`, or b) use Monte Carlo to
estimate the variances. See paper "Variance estimation when using inverse probability of treatment weighting (IPTW) with survival analysis"

  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/coxph_fitter.py:1607: ConvergenceWarning: Newton-Raphson convergence completed successfully but norm(delta) is still high, 0.359. This may imply non-unique solutions to the maximum likelihood. Perhaps there is collinearity or complete separation in the dataset?

  warnings.warn(
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/lifelines/fitters/coxph_fitter.py:1356: Sta

<lifelines.CoxPHFitter: fitted with 3 total observations, 2 right-censored observations>

In [29]:
# Display the summary of each model
print("Model Summary for Event Type 1:")
print(cox_model_event_1.summary)
print(cox_model_event_1.AIC_partial_)
print(cox_model_event_1.concordance_index_)

print("\nModel Summary for Event Type 2:")
print(cox_model_event_2.summary)

Model Summary for Event Type 1:
               coef     exp(coef)  se(coef)  coef lower 95%  coef upper 95%  \
covariate                                                                     
gender    -0.355755      0.700644  0.054498       -0.462570       -0.248941   
dm        -0.143351      0.866450  0.058294       -0.257605       -0.029097   
ht        -0.035341      0.965276  0.057227       -0.147503        0.076821   
sprint    -0.164435      0.848373  0.059644       -0.281335       -0.047536   
a1c        0.048643      1.049846  0.768265       -1.457129        1.554416   
po4       -1.333574      0.263534  0.474542       -2.263660       -0.403488   
UACR_mg_g  0.768474      2.156473  0.353309        0.076002        1.460947   
Cr         9.501827  13384.161837  0.323643        8.867499       10.136156   
age        0.144763      1.155766  0.182369       -0.212673        0.502199   
alb        0.831519      2.296804  0.401427        0.044736        1.618302   
ca        -2.703071 

In [30]:
# Step 2: Predict individual cumulative hazards for each event type
cumulative_hazard_event_1 = cox_model_event_1.predict_cumulative_hazard(X_train_transformed[FEATURE_COLS])
cumulative_hazard_event_2 = cox_model_event_2.predict_cumulative_hazard(X_train_transformed[FEATURE_COLS])

In [31]:
# Step 3: Compute overall survival for each individual
# Overall survival: S(t) = exp(- (H1(t) + H2(t)))
overall_survival = np.exp(-(cumulative_hazard_event_1 + cumulative_hazard_event_2))

In [65]:
# Step 4: Calculate CIF for each event type
# CIF_k(t) = ∫ h_k(u) * S(u) du (approximated as cumulative sum)
cif_event_1 = (cumulative_hazard_event_1 * overall_survival).cumsum(axis=0)
cif_event_2 = (cumulative_hazard_event_2 * overall_survival).cumsum(axis=0)
cif_event_1_normalized = cif_event_1 / cif_event_1.iloc[-1].max()
cif_event_2_normalized = cif_event_2 / cif_event_2.iloc[-1].max()


In [66]:
cif_event_1_normalized.loc[TIME_GRID[1:], 8]

365.0     0.001461
730.0     0.005751
1095.0    0.012262
1460.0    0.020153
1825.0    0.027249
Name: 8, dtype: float64

In [34]:
# Step 5: Format and display the CIF predictions
cif_predictions = pd.DataFrame({
    "CIF_Event_1": cif_event_1.iloc[:, -1],  # CIF for Event 1 at the last time point
    "CIF_Event_2": cif_event_2.iloc[:, -1]   # CIF for Event 2 at the last time point
}, index=X_train_transformed.index)

In [35]:
cif_predictions

,CIF_Event_1,CIF_Event_2
0,NaN,NaN
1,0.000000,0.0
2,0.000000,0.0
3,0.000000,0.0
4,0.000227,0.0
...,...,...
396419,NaN,NaN
396420,NaN,NaN
396421,NaN,NaN
396422,NaN,NaN
